In [1]:
#import sys
#!{sys.executable} -m pip install --upgrade "scikit-learn>=1.6.1,<1.7" category_encoders shap lyft_fugue

In [3]:
#usual packages
import sklearn
import numpy as np
from IPython.display import IFrame
from sklearn.model_selection import GroupKFold
from datetime import date
import pandas as pd
from pandas import DataFrame
import subprocess
import sys

#lyft packages
import lyft_data_toolkit.db.write.df
from lyft_data_toolkit.db.read import Compute
from lyft_data_toolkit.db.read import Query
from lyft_data_toolkit.db.read.df import to_dataframe  
from lyft_data_toolkit.v1_to_v3 import hive
from lyft_data_toolkit.v1_to_v3 import presto
from lyft_data_toolkit.v1_to_v3 import spark
from lyft_data_toolkit.v1_to_v3 import trino

#other packages
import matplotlib.pyplot as plt
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

### 1. Run/change parameters

In [4]:
########### parameters 
launch_date = "2025-08-04"
report_start_dt = "2025-10-27"
report_end_dt = "2025-11-02"

In [5]:
lift_lookup = pd.read_csv("weekly_lift_data.csv")
### insert in new on demand data 
lift_lookup = pd.concat([
    lift_lookup,
    pd.DataFrame([{
        "week_num": 40, #######
        "year_num": 2025, ###
        "starting_week_date": "2025-09-29", #######
        "pre_existing_rider_lift": 0.0023, #########
        "new_rider_pp_lift": 1.833 #######
    }])
], ignore_index=True)

lift_lookup = lift_lookup.sort_values("starting_week_date").drop_duplicates(ignore_index=True).reset_index(drop=True)
lift_lookup
lift_lookup.to_csv("weekly_lift_data.csv",index =False)

In [6]:
lift_lookup

,week_num,year_num,starting_week_date,pre_existing_rider_lift,new_rider_pp_lift
0,32,2025,2025-08-04,0.0023,1.8330
1,33,2025,2025-08-11,0.0023,1.8330
2,34,2025,2025-08-18,0.0023,1.8330
3,35,2025,2025-08-25,0.0023,1.8330
4,36,2025,2025-09-01,0.0023,1.8330
5,37,2025,2025-09-08,0.0023,1.8330
6,38,2025,2025-09-15,0.0023,1.8330
7,39,2025,2025-09-22,0.0023,1.8330
8,40,2025,2025-09-29,0.0023,1.8330
9,41,2025,2025-10-06,0.0130,1.2870


### 2. Run Prediction

In [8]:

SQL ="""
--higherlimit
WITH pre_final_table AS (
SELECT DISTINCT 
       sub1.rider_lyft_id,
       sub1.business_program_user_id,
       sub1.business_program_type,
       sub1.business_program_type_cleaned,
       sub1.lyft_signed_up_at,
       sub1.bp_created_at,
       sub1.bp_activated_at,
       sub1.user_type,
       sub1.sub_user_type,
       sub2.first_ride_finished_at
  FROM shuninga.causal_analysis_bp_raw_data_step_4 sub1
  LEFT JOIN coco.dim_user sub2 
    ON sub1.rider_lyft_id = sub2.user_lyft_id
 WHERE sub1.is_business_ride = TRUE 
) 

,pre_final_table_v2 AS (
SELECT rider_lyft_id,
       business_program_user_id,
       business_program_type,
       business_program_type_cleaned,
       lyft_signed_up_at,
       bp_created_at,
       bp_activated_at,
       user_type,
       sub_user_type,
       first_ride_finished_at,
       ROW_NUMBER() OVER(PARTITION BY rider_lyft_id ORDER BY bp_activated_at DESC) rnk 
  FROM pre_final_table 
 WHERE TRUE
)

SELECT DATE_TRUNC('week',CAST(bp_activated_at AS DATE)) week,
       business_program_type_cleaned,
       CASE WHEN DATE_DIFF('day',first_ride_finished_at,bp_activated_at) > 0 THEN 'PP' ELSE 'NP' END AS activation_type, -- proxy for NP/PP
       COUNT(DISTINCT rider_lyft_id) count_activations
  FROM pre_final_table_v2
 WHERE TRUE 
   AND rnk = 1
 GROUP BY 1,2,3
"""  
df = pd.DataFrame()
df = trino.query(SQL, scheduled=True)

{"ts": "2025-11-10T20:51:00.175Z", "uuid": "8f9f64c6-5d03-4703-8f95-6b54645018fd", "name": "lyft_data_toolkit.v1_to_v3.trino", "lvlname": "WARNING", "lvlno": 30, "pid": 1246, "call_site": "code/venvs/venv/lib/python3.10/site-packages/lyft_data_toolkit/v1_to_v3/trino.py:26", "revision": "b8758af9c9", "msg": "DEPRECATION WARNING: trino.query method is outdated.Please use the new query function in the lyft_data_toolkit package."}


Created Mozart command
{'mozart_id': 565791303, 'backend': 'presto', 'native_command_id': '', 'job_url': 'https://sql.lyft.net/databases/mozart/queries/565791303', 'query': "--higherlimit\nWITH pre_final_table AS (\nSELECT DISTINCT \n       sub1.rider_lyft_id,\n       sub1.business_program_user_id,\n       sub1.business_program_type,\n       sub1.business_program_type_cleaned,\n       sub1.lyft_signed_up_at,\n       sub1.bp_created_at,\n       sub1.bp_activated_at,\n       sub1.user_type,\n       sub1.sub_user_type,\n       sub2.first_ride_finished_at\n  FROM shuninga.causal_analysis_bp_raw_data_step_4 sub1\n  LEFT JOIN coco.dim_user sub2 \n    ON sub1.rider_lyft_id = sub2.user_lyft_id\n WHERE sub1.is_business_ride = TRUE \n) \n\n,pre_final_table_v2 AS (\nSELECT rider_lyft_id,\n       business_program_user_id,\n       business_program_type,\n       business_program_type_cleaned,\n       lyft_signed_up_at,\n       bp_created_at,\n       bp_activated_at,\n       user_type,\n       sub_us

In [ ]:
df.columns

In [ ]:
#pivot table to have columns for managed vs organic activations
df_pivot = df.pivot_table(
    index=["week", "activation_type"],
    columns="business_program_type_cleaned",
    values="count_activations",
    fill_value=0
).reset_index()

print(df_pivot)

In [ ]:
np_df = df_pivot[df_pivot.activation_type == "NP"]
pp_df = df_pivot[df_pivot.activation_type == "PP"]

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from joblib import load

# launch_date = "2025-08-04"

np_df = df_pivot[df_pivot["activation_type"] == "NP"].copy()
pp_df = df_pivot[df_pivot["activation_type"] == "PP"].copy()

def add_lags(df):
    df = df.sort_values("week").reset_index(drop=True)
    df["organic_lag1"] = df["organic"].shift(1).fillna(method='bfill')
    df["organic_lag2"] = df["organic"].shift(2).fillna(method='bfill')
    return df

np_df = add_lags(np_df)
pp_df = add_lags(pp_df)

def predict_post_rewards(df, model_file, use_lag2=True):
    post_df = df[df["week"] >= launch_date].copy()
    if post_df.empty:
        print(f"No post-rewards data for {model_file}")
        return None
    
    #features = ["organic", "organic_lag1"]
    #if use_lag2:
        #features.append("organic_lag2")
    
    X_post = post_df["organic"]
    
    model = load(model_file)
    
    post_df["predicted_managed"] = model.predict(
        start=len(df[df["week"] < launch_date]),
        end=len(df)-1,
        exog=X_post
    )
    
    return post_df[["week", "managed", "predicted_managed"]]

np_post_preds = predict_post_rewards(np_df, "arimax_model_np.pkl", use_lag2=False)
pp_post_preds = predict_post_rewards(pp_df, "arimax_model_pp.pkl", use_lag2=True)

if np_post_preds is not None:
    np_post_preds.to_csv("np_post_rewards_predictions.csv", index=False)
    print("NP post-rewards predictions exported to np_post_rewards_predictions.csv")

if pp_post_preds is not None:
    pp_post_preds.to_csv("pp_post_rewards_predictions.csv", index=False)
    print("PP post-rewards predictions exported to pp_post_rewards_predictions.csv")

print("NP Post-Rewards Predictions:")
print(np_post_preds)

print("PP Post-Rewards Predictions:")
print(pp_post_preds)


### 3. calculate additional acitivation percentage - get incremental part into dataframe

In [ ]:
np_preds = np_post_preds.reset_index(drop= True)
pp_preds = pp_post_preds.reset_index(drop = True)

In [ ]:
np_preds['addl_activations'] = ((np_preds['managed'] - np_preds['predicted_managed']).clip(lower=0)).astype(int)
pp_preds['addl_activations'] = ((pp_preds['managed'] - pp_preds['predicted_managed']).clip(lower=0)).astype(int)

In [ ]:
np_preds['perc_addl_activations'] = np_preds['addl_activations']/np_preds['managed']
pp_preds['perc_addl_activations'] = pp_preds['addl_activations']/pp_preds['managed']

In [ ]:
np_preds


### 5. to build pre existing, new user pp, np rides table

#### 5.1 get pre existing data

In [18]:
pre_existing_sql  = f"""
-- higherlimit
SELECT week(requested_at) as week_num, year(requested_at) as year_num, 
DATE_TRUNC('week',CAST(requested_at AS DATE)) starting_week_date, 
date_diff('week',date'2025-08-04', requested_at) as post_reward_week_num,
'pre_existing' as user_type,
count(distinct ride_id) as num_rides_pre_existing,
sum(count(distinct ride_id)) over (order by date_diff('week',date'2025-08-04', requested_at) asc) as cum_rides_pre_existing
FROM shuninga.causal_analysis_bp_raw_data_step_4 
 WHERE user_type = 'pre-existing-user'
   AND most_used_business_program_type_cleaned = 'managed' 
  and date(requested_at) between date'{launch_date}' and date'{report_end_dt}'
  group by 1,2,3,4,5
  order by 1,2
  """ 
pre_exisiting_rides = pd.DataFrame()
pre_exisiting_rides = trino.query(pre_existing_sql, scheduled= True)

{"ts": "2025-11-10T18:45:48.180Z", "uuid": "6287a29e-883b-4b07-99ac-9917801419f2", "name": "lyft_data_toolkit.v1_to_v3.trino", "lvlname": "WARNING", "lvlno": 30, "pid": 97, "call_site": "code/venvs/venv/lib/python3.10/site-packages/lyft_data_toolkit/v1_to_v3/trino.py:26", "revision": "b8758af9c9", "msg": "DEPRECATION WARNING: trino.query method is outdated.Please use the new query function in the lyft_data_toolkit package."}


Created Mozart command
{'mozart_id': 565742148, 'backend': 'presto', 'native_command_id': '', 'job_url': 'https://sql.lyft.net/databases/mozart/queries/565742148', 'query': "-- higherlimit\nSELECT week(requested_at) as week_num, year(requested_at) as year_num, \nDATE_TRUNC('week',CAST(requested_at AS DATE)) starting_week_date, \ndate_diff('week',date'2025-08-04', requested_at) as post_reward_week_num,\n'pre_existing' as user_type,\ncount(distinct ride_id) as num_rides_pre_existing,\nsum(count(distinct ride_id)) over (order by date_diff('week',date'2025-08-04', requested_at) asc) as cum_rides_pre_existing\nFROM shuninga.causal_analysis_bp_raw_data_step_4 \n WHERE user_type = 'pre-existing-user'\n   AND most_used_business_program_type_cleaned = 'managed' \n  and date(requested_at) between date'2025-08-04' and date'2025-11-02'\n  group by 1,2,3,4,5\n  order by 1,2"}
s3://mozartgateway-production-iad/mz/cmd/results/565742148/


In [19]:
pre_exisiting_rides.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   week_num                13 non-null     int64         
 1   year_num                13 non-null     int64         
 2   starting_week_date      13 non-null     datetime64[ns]
 3   post_reward_week_num    13 non-null     int64         
 4   user_type               13 non-null     object        
 5   num_rides_pre_existing  13 non-null     int64         
 6   cum_rides_pre_existing  13 non-null     int64         
dtypes: datetime64[ns](1), int64(5), object(1)
memory usage: 856.0+ bytes


In [20]:
lift_lookup["starting_week_date"] = pd.to_datetime(lift_lookup["starting_week_date"])
pre_exisiting_rides = pd.merge(
    pre_exisiting_rides,
    lift_lookup[["starting_week_date", "pre_existing_rider_lift"]],
    on="starting_week_date",
    how="left" 
)
pre_exisiting_rides.drop_duplicates(ignore_index=True)

,week_num,year_num,starting_week_date,post_reward_week_num,user_type,num_rides_pre_existing,cum_rides_pre_existing,pre_existing_rider_lift
0,32,2025,2025-08-04,0,pre_existing,71141,71141,0.0023
1,33,2025,2025-08-11,1,pre_existing,69991,141132,0.0023
2,34,2025,2025-08-18,2,pre_existing,69816,210948,0.0023
3,35,2025,2025-08-25,3,pre_existing,67417,278365,0.0023
4,36,2025,2025-09-01,4,pre_existing,63875,342240,0.0023
5,37,2025,2025-09-08,5,pre_existing,78497,420737,0.0023
6,38,2025,2025-09-15,6,pre_existing,81658,502395,0.0023
7,39,2025,2025-09-22,7,pre_existing,76792,579187,0.0023
8,40,2025,2025-09-29,8,pre_existing,74178,653365,0.0023
9,41,2025,2025-10-06,9,pre_existing,79520,732885,0.0130


In [66]:
pre_exisiting_rides['pre_existing_incr_rides'] =  (1-1/(1+pre_exisiting_rides['pre_existing_rider_lift']))*pre_exisiting_rides['num_rides_pre_existing']
pre_exisiting_rides

,week_num,year_num,starting_week_date,post_reward_week_num,user_type,num_rides_pre_existing,cum_rides_pre_existing,pre_existing_rider_lift,incr_rides,pre_existing_incr_rides
0,32,2025,2025-08-04,0,pre_existing,71141,71141,0.0023,163.248828,163.248828
1,33,2025,2025-08-11,1,pre_existing,69991,141132,0.0023,160.609897,160.609897
2,34,2025,2025-08-18,2,pre_existing,69816,210948,0.0023,160.208321,160.208321
3,35,2025,2025-08-25,3,pre_existing,67417,278365,0.0023,154.703282,154.703282
4,36,2025,2025-09-01,4,pre_existing,63875,342240,0.0023,146.575377,146.575377
5,37,2025,2025-09-08,5,pre_existing,78497,420737,0.0023,180.128804,180.128804
6,38,2025,2025-09-15,6,pre_existing,81658,502395,0.0023,187.382420,187.382420
7,39,2025,2025-09-22,7,pre_existing,76792,579187,0.0023,176.216303,176.216303
8,40,2025,2025-09-29,8,pre_existing,74178,653365,0.0023,170.217899,170.217899
9,41,2025,2025-10-06,9,pre_existing,79520,732885,0.0130,1020.493583,1020.493583


#### 5.2 get np and pp data

In [25]:
########## pp new user 
# np_preds 
# pp_preds
np_preds

business_program_type_cleaned,week,managed,predicted_managed,addl_activations,perc_addl_activations
0,2025-08-04,134,133.393125,0,0.000000
1,2025-08-11,120,144.028087,0,0.000000
2,2025-08-18,131,141.851859,0,0.000000
3,2025-08-25,92,121.021250,0,0.000000
4,2025-09-01,109,110.594195,0,0.000000
5,2025-09-08,153,160.600157,0,0.000000
6,2025-09-15,172,162.041984,9,0.052326
7,2025-09-22,151,143.659792,7,0.046358
8,2025-09-29,170,126.124932,43,0.252941
9,2025-10-06,195,151.968766,43,0.220513


In [67]:
new_user_pp_sql = f"""
WITH week_bounds AS (
  SELECT 
    week_start,
    DATE_ADD('day', 6, week_start) AS week_end
  FROM UNNEST(SEQUENCE(DATE '2025-08-04', DATE'{report_end_dt}', INTERVAL '7' DAY)) AS t(week_start)
)

SELECT
  week(wb.week_start) as week_num,
  year(wb.week_start) as year_num,
  wb.week_start as starting_week_date,
  date_diff('week',date'2025-08-04', week_start) as post_reward_week_num,
  'new user pp' as user_type,
  COUNT(DISTINCT ride_id) AS num_rides_new_pp,
  sum(count(distinct ride_id)) over (order by date_diff('week',date'2025-08-04', week_start) asc) as cum_rides_new_pp
FROM week_bounds wb
LEFT JOIN shuninga.causal_analysis_bp_raw_data_step_4 d
  ON DATE(d.requested_at) BETWEEN date(wb.week_start) AND date(wb.week_end)
  AND d.user_type = 'new-user-PP'
  AND d.most_used_business_program_type_cleaned = 'managed'
  AND d.most_used_bp_activated_at BETWEEN DATE'2025-08-04' AND date(wb.week_end)
GROUP BY 1,2,3,4,5
ORDER BY 3
"""
new_pp_rides = pd.DataFrame()
new_pp_rides = trino.query(new_user_pp_sql,scheduled =True)
new_pp_rides

{"ts": "2025-11-10T19:41:39.715Z", "uuid": "56ff7602-a95c-411e-84ee-3dddce6a6a34", "name": "lyft_data_toolkit.v1_to_v3.trino", "lvlname": "WARNING", "lvlno": 30, "pid": 97, "call_site": "code/venvs/venv/lib/python3.10/site-packages/lyft_data_toolkit/v1_to_v3/trino.py:26", "revision": "b8758af9c9", "msg": "DEPRECATION WARNING: trino.query method is outdated.Please use the new query function in the lyft_data_toolkit package."}
/code/venvs/venv/lib/python3.10/site-packages/lyft_stats/client.py:330: UserWarning: [lyft_stats.client] stats dropped due to err=ConnectionResetError(104, 'Connection reset by peer')
  warnings.warn(f"[{__name__}] stats dropped due to {err=}")


Created Mozart command
{'mozart_id': 565764728, 'backend': 'presto', 'native_command_id': '', 'job_url': 'https://sql.lyft.net/databases/mozart/queries/565764728', 'query': "WITH week_bounds AS (\n  SELECT \n    week_start,\n    DATE_ADD('day', 6, week_start) AS week_end\n  FROM UNNEST(SEQUENCE(DATE '2025-08-04', DATE'2025-11-02', INTERVAL '7' DAY)) AS t(week_start)\n)\n\nSELECT\n  week(wb.week_start) as week_num,\n  year(wb.week_start) as year_num,\n  wb.week_start as starting_week_date,\n  date_diff('week',date'2025-08-04', week_start) as post_reward_week_num,\n  'new user pp' as user_type,\n  COUNT(DISTINCT ride_id) AS num_rides_new_pp,\n  sum(count(distinct ride_id)) over (order by date_diff('week',date'2025-08-04', week_start) asc) as cum_rides_new_pp\nFROM week_bounds wb\nLEFT JOIN shuninga.causal_analysis_bp_raw_data_step_4 d\n  ON DATE(d.requested_at) BETWEEN date(wb.week_start) AND date(wb.week_end)\n  AND d.user_type = 'new-user-PP'\n  AND d.most_used_business_program_type_cl

/code/venvs/venv/lib/python3.10/site-packages/lyft_stats/client.py:347: UserWarning: Attempting to establish a new TCP stats connection
  warnings.warn("Attempting to establish a new TCP stats connection")


s3://mozartgateway-production-iad/mz/cmd/results/565764728/


,week_num,year_num,starting_week_date,post_reward_week_num,user_type,num_rides_new_pp,cum_rides_new_pp
0,32,2025,2025-08-04,0,new user pp,2514,2514
1,33,2025,2025-08-11,1,new user pp,3120,5634
2,34,2025,2025-08-18,2,new user pp,3787,9421
3,35,2025,2025-08-25,3,new user pp,4121,13542
4,36,2025,2025-09-01,4,new user pp,4062,17604
5,37,2025,2025-09-08,5,new user pp,6336,23940
6,38,2025,2025-09-15,6,new user pp,7777,31717
7,39,2025,2025-09-22,7,new user pp,7533,39250
8,40,2025,2025-09-29,8,new user pp,7691,46941
9,41,2025,2025-10-06,9,new user pp,9493,56434


In [33]:
pp_preds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   week                   14 non-null     datetime64[ns]
 1   managed                14 non-null     int64         
 2   predicted_managed      14 non-null     float64       
 3   addl_activations       14 non-null     int64         
 4   perc_addl_activations  14 non-null     float64       
dtypes: datetime64[ns](1), float64(2), int64(2)
memory usage: 688.0 bytes


In [68]:
new_pp_rides = pd.merge(new_pp_rides,
                        pp_preds[["week","perc_addl_activations"]],
                        left_on = "starting_week_date",
                        right_on = "week",
                        how = "left"
                       ) 

new_pp_rides = pd.merge(new_pp_rides,
                        lift_lookup[['starting_week_date','pre_existing_rider_lift','new_rider_pp_lift']],
                        on = 'starting_week_date',
                        how = 'left')
new_pp_rides

,week_num,year_num,starting_week_date,post_reward_week_num,user_type,num_rides_new_pp,cum_rides_new_pp,week,perc_addl_activations,pre_existing_rider_lift,new_rider_pp_lift
0,32,2025,2025-08-04,0,new user pp,2514,2514,2025-08-04,0.000000,0.0023,1.8330
1,33,2025,2025-08-11,1,new user pp,3120,5634,2025-08-11,0.000000,0.0023,1.8330
2,34,2025,2025-08-18,2,new user pp,3787,9421,2025-08-18,0.053853,0.0023,1.8330
3,35,2025,2025-08-25,3,new user pp,4121,13542,2025-08-25,0.000000,0.0023,1.8330
4,36,2025,2025-09-01,4,new user pp,4062,17604,2025-09-01,0.000000,0.0023,1.8330
5,37,2025,2025-09-08,5,new user pp,6336,23940,2025-09-08,0.095238,0.0023,1.8330
6,38,2025,2025-09-15,6,new user pp,7777,31717,2025-09-15,0.115593,0.0023,1.8330
7,39,2025,2025-09-22,7,new user pp,7533,39250,2025-09-22,0.077147,0.0023,1.8330
8,40,2025,2025-09-29,8,new user pp,7691,46941,2025-09-29,0.093651,0.0023,1.8330
9,41,2025,2025-10-06,9,new user pp,9493,56434,2025-10-06,0.169141,0.0130,1.2870


In [69]:
### calculate incremental rides
new_pp_rides['pp_counterfactual_rides'] = new_pp_rides['num_rides_new_pp']*(1-new_pp_rides['perc_addl_activations'])
new_pp_rides['pp_addl_rides'] = new_pp_rides['num_rides_new_pp']- new_pp_rides['pp_counterfactual_rides'] 
new_pp_rides['pp_incre_rides_from_counterfactual'] = (1-1/(1+new_pp_rides['pre_existing_rider_lift']))*new_pp_rides['pp_counterfactual_rides']
new_pp_rides['pp_incre_rides_from_addl'] = (1-1/(1+new_pp_rides['new_rider_pp_lift']))*new_pp_rides['pp_addl_rides'] 

In [70]:
new_pp_rides

,week_num,year_num,starting_week_date,post_reward_week_num,user_type,num_rides_new_pp,cum_rides_new_pp,week,perc_addl_activations,pre_existing_rider_lift,new_rider_pp_lift,pp_counterfactual_rides,pp_addl_rides,pp_incre_rides_from_counterfactual,pp_incre_rides_from_addl
0,32,2025,2025-08-04,0,new user pp,2514,2514,2025-08-04,0.000000,0.0023,1.8330,2514.000000,0.000000,5.768931,0.000000
1,33,2025,2025-08-11,1,new user pp,3120,5634,2025-08-11,0.000000,0.0023,1.8330,3120.000000,0.000000,7.159533,0.000000
2,34,2025,2025-08-18,2,new user pp,3787,9421,2025-08-18,0.053853,0.0023,1.8330,3583.060481,203.939519,8.222128,131.952396
3,35,2025,2025-08-25,3,new user pp,4121,13542,2025-08-25,0.000000,0.0023,1.8330,4121.000000,0.000000,9.456550,0.000000
4,36,2025,2025-09-01,4,new user pp,4062,17604,2025-09-01,0.000000,0.0023,1.8330,4062.000000,0.000000,9.321161,0.000000
5,37,2025,2025-09-08,5,new user pp,6336,23940,2025-09-08,0.095238,0.0023,1.8330,5732.571429,603.428571,13.154659,390.428723
6,38,2025,2025-09-15,6,new user pp,7777,31717,2025-09-15,0.115593,0.0023,1.8330,6878.033113,898.966887,15.783175,581.647125
7,39,2025,2025-09-22,7,new user pp,7533,39250,2025-09-22,0.077147,0.0023,1.8330,6951.851528,581.148472,15.952568,376.013113
8,40,2025,2025-09-29,8,new user pp,7691,46941,2025-09-29,0.093651,0.0023,1.8330,6970.731746,720.268254,15.995892,466.026018
9,41,2025,2025-10-06,9,new user pp,9493,56434,2025-10-06,0.169141,0.0130,1.2870,7887.340865,1605.659135,101.219577,903.578184


#### get new user np data

In [73]:
new_user_np_sql=f"""
WITH week_bounds AS (
  SELECT 
    week_start,
    DATE_ADD('day', 6, week_start) AS week_end
  FROM UNNEST(SEQUENCE(DATE '2025-08-04', DATE'{report_end_dt}', INTERVAL '7' DAY)) AS t(week_start)
)

SELECT
  week(wb.week_start) as week_num,
  year(wb.week_start) as year_num,
  wb.week_start as starting_week_date,
  date_diff('week',date'2025-08-04', week_start) as post_reward_week_num,
  'new user np' as user_type,
  COUNT(DISTINCT ride_id) AS num_rides_new_np,
  sum(count(distinct ride_id)) over (order by date_diff('week',date'2025-08-04', week_start) asc) as cum_rides_new_np
FROM week_bounds wb
LEFT JOIN shuninga.causal_analysis_bp_raw_data_step_4 d
  ON DATE(d.requested_at) BETWEEN date(wb.week_start) AND date(wb.week_end)
  AND d.user_type = 'new-user-NP'
  AND d.most_used_business_program_type_cleaned = 'managed'
  AND d.most_used_bp_activated_at BETWEEN DATE'2025-08-04' AND date(wb.week_end)
GROUP BY 1,2,3,4,5
ORDER BY 3
"""
new_np_rides = pd.DataFrame()
new_np_rides = trino.query(new_user_np_sql,scheduled =True)
new_np_rides

{"ts": "2025-11-10T19:44:28.213Z", "uuid": "b9cd16de-badf-4812-87a0-51dbcddd26ea", "name": "lyft_data_toolkit.v1_to_v3.trino", "lvlname": "WARNING", "lvlno": 30, "pid": 97, "call_site": "code/venvs/venv/lib/python3.10/site-packages/lyft_data_toolkit/v1_to_v3/trino.py:26", "revision": "b8758af9c9", "msg": "DEPRECATION WARNING: trino.query method is outdated.Please use the new query function in the lyft_data_toolkit package."}


Created Mozart command
{'mozart_id': 565765588, 'backend': 'presto', 'native_command_id': '', 'job_url': 'https://sql.lyft.net/databases/mozart/queries/565765588', 'query': "WITH week_bounds AS (\n  SELECT \n    week_start,\n    DATE_ADD('day', 6, week_start) AS week_end\n  FROM UNNEST(SEQUENCE(DATE '2025-08-04', DATE'2025-11-02', INTERVAL '7' DAY)) AS t(week_start)\n)\n\nSELECT\n  week(wb.week_start) as week_num,\n  year(wb.week_start) as year_num,\n  wb.week_start as starting_week_date,\n  date_diff('week',date'2025-08-04', week_start) as post_reward_week_num,\n  'new user np' as user_type,\n  COUNT(DISTINCT ride_id) AS num_rides_new_np,\n  sum(count(distinct ride_id)) over (order by date_diff('week',date'2025-08-04', week_start) asc) as cum_rides_new_np\nFROM week_bounds wb\nLEFT JOIN shuninga.causal_analysis_bp_raw_data_step_4 d\n  ON DATE(d.requested_at) BETWEEN date(wb.week_start) AND date(wb.week_end)\n  AND d.user_type = 'new-user-NP'\n  AND d.most_used_business_program_type_cl

,week_num,year_num,starting_week_date,post_reward_week_num,user_type,num_rides_new_np,cum_rides_new_np
0,32,2025,2025-08-04,0,new user np,173,173
1,33,2025,2025-08-11,1,new user np,356,529
2,34,2025,2025-08-18,2,new user np,480,1009
3,35,2025,2025-08-25,3,new user np,478,1487
4,36,2025,2025-09-01,4,new user np,552,2039
5,37,2025,2025-09-08,5,new user np,741,2780
6,38,2025,2025-09-15,6,new user np,943,3723
7,39,2025,2025-09-22,7,new user np,945,4668
8,40,2025,2025-09-29,8,new user np,957,5625
9,41,2025,2025-10-06,9,new user np,1233,6858


In [74]:
new_np_rides = pd.merge(new_np_rides,
                        np_preds[["week","perc_addl_activations"]],
                        left_on = "starting_week_date",
                        right_on = "week",
                        how = "left"
                       ) 
new_np_rides = pd.merge(new_np_rides,
                        lift_lookup[['starting_week_date','pre_existing_rider_lift']],
                        on = 'starting_week_date',
                        how = 'left')

In [63]:
new_np_rides

,week_num,year_num,starting_week_date,post_reward_week_num,user_type,num_rides_new_np,cum_rides_new_np,week,perc_addl_activations,pre_existing_rider_lift
0,32,2025,2025-08-04,0,new user np,173,173,2025-08-04,0.000000,0.0023
1,33,2025,2025-08-11,1,new user np,356,529,2025-08-11,0.000000,0.0023
2,34,2025,2025-08-18,2,new user np,480,1009,2025-08-18,0.000000,0.0023
3,35,2025,2025-08-25,3,new user np,478,1487,2025-08-25,0.000000,0.0023
4,36,2025,2025-09-01,4,new user np,552,2039,2025-09-01,0.000000,0.0023
5,37,2025,2025-09-08,5,new user np,741,2780,2025-09-08,0.000000,0.0023
6,38,2025,2025-09-15,6,new user np,943,3723,2025-09-15,0.052326,0.0023
7,39,2025,2025-09-22,7,new user np,945,4668,2025-09-22,0.046358,0.0023
8,40,2025,2025-09-29,8,new user np,957,5625,2025-09-29,0.252941,0.0023
9,41,2025,2025-10-06,9,new user np,1233,6858,2025-10-06,0.220513,0.0130


In [75]:
### calculate incremental rides
new_np_rides['np_counterfactual_rides'] = new_np_rides['num_rides_new_np']*(1-new_np_rides['perc_addl_activations'])
new_np_rides['np_addl_rides'] = new_np_rides['num_rides_new_np']- new_np_rides['np_counterfactual_rides'] 
new_np_rides['np_incre_rides_from_counterfactual'] = (1-1/(1+new_np_rides['pre_existing_rider_lift']))*new_np_rides['np_counterfactual_rides']
new_np_rides['np_incre_rides_from_addl'] = new_np_rides['np_addl_rides']
new_np_rides

,week_num,year_num,starting_week_date,post_reward_week_num,user_type,num_rides_new_np,cum_rides_new_np,week,perc_addl_activations,pre_existing_rider_lift,np_counterfactual_rides,np_addl_rides,np_incre_rides_from_counterfactual,np_incre_rides_from_addl
0,32,2025,2025-08-04,0,new user np,173,173,2025-08-04,0.000000,0.0023,173.000000,0.000000,0.396987,0.000000
1,33,2025,2025-08-11,1,new user np,356,529,2025-08-11,0.000000,0.0023,356.000000,0.000000,0.816921,0.000000
2,34,2025,2025-08-18,2,new user np,480,1009,2025-08-18,0.000000,0.0023,480.000000,0.000000,1.101467,0.000000
3,35,2025,2025-08-25,3,new user np,478,1487,2025-08-25,0.000000,0.0023,478.000000,0.000000,1.096877,0.000000
4,36,2025,2025-09-01,4,new user np,552,2039,2025-09-01,0.000000,0.0023,552.000000,0.000000,1.266687,0.000000
5,37,2025,2025-09-08,5,new user np,741,2780,2025-09-08,0.000000,0.0023,741.000000,0.000000,1.700389,0.000000
6,38,2025,2025-09-15,6,new user np,943,3723,2025-09-15,0.052326,0.0023,893.656977,49.343023,2.050694,49.343023
7,39,2025,2025-09-22,7,new user np,945,4668,2025-09-22,0.046358,0.0023,901.192053,43.807947,2.067985,43.807947
8,40,2025,2025-09-29,8,new user np,957,5625,2025-09-29,0.252941,0.0023,714.935294,242.064706,1.640578,242.064706
9,41,2025,2025-10-06,9,new user np,1233,6858,2025-10-06,0.220513,0.0130,961.107692,271.892308,12.334057,271.892308


In [65]:
pre_exisiting_rides.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13 entries, 0 to 12
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   week_num                 13 non-null     int64         
 1   year_num                 13 non-null     int64         
 2   starting_week_date       13 non-null     datetime64[ns]
 3   post_reward_week_num     13 non-null     int64         
 4   user_type                13 non-null     object        
 5   num_rides_pre_existing   13 non-null     int64         
 6   cum_rides_pre_existing   13 non-null     int64         
 7   pre_existing_rider_lift  13 non-null     float64       
 8   incr_rides               13 non-null     float64       
dtypes: datetime64[ns](1), float64(2), int64(5), object(1)
memory usage: 1.0+ KB


In [90]:
#### get final result with all incrementality

incremental_all_rides = pd.merge(
                        pre_exisiting_rides[['week_num','year_num','starting_week_date','post_reward_week_num','num_rides_pre_existing','cum_rides_pre_existing','pre_existing_rider_lift','pre_existing_incr_rides']],
                        new_pp_rides[['starting_week_date','num_rides_new_pp','cum_rides_new_pp','new_rider_pp_lift','pp_counterfactual_rides','pp_addl_rides','pp_incre_rides_from_counterfactual','pp_incre_rides_from_addl']],
                        on = 'starting_week_date',
                        how = 'left'
)

incremental_all_rides = pd.merge(
    incremental_all_rides,
    new_np_rides[['starting_week_date','num_rides_new_np','cum_rides_new_np','np_counterfactual_rides','np_addl_rides','np_incre_rides_from_counterfactual','np_incre_rides_from_addl']],
    on = 'starting_week_date',
    how = 'left'
)
incremental_all_rides['total_incre_rides']= incremental_all_rides['pre_existing_incr_rides']+ incremental_all_rides['pp_incre_rides_from_counterfactual']+ incremental_all_rides['pp_incre_rides_from_addl']+incremental_all_rides['np_incre_rides_from_counterfactual']+incremental_all_rides['np_incre_rides_from_addl']
incremental_all_rides['cum_total_incre_rides'] = incremental_all_rides['total_incre_rides'].cumsum()
incremental_all_rides['total_observed_rides'] = incremental_all_rides['num_rides_pre_existing']+incremental_all_rides['num_rides_new_pp']+incremental_all_rides['num_rides_new_np']
incremental_all_rides['cum_total_observed_rides'] = incremental_all_rides['total_observed_rides'].cumsum()
incremental_all_rides['cum_baseline_rides'] = incremental_all_rides['cum_total_observed_rides']-incremental_all_rides['cum_total_incre_rides']
incremental_all_rides['progress_t_target'] = incremental_all_rides['cum_total_incre_rides']/713894
incremental_all_rides

,week_num,year_num,starting_week_date,post_reward_week_num,num_rides_pre_existing,cum_rides_pre_existing,pre_existing_rider_lift,pre_existing_incr_rides,num_rides_new_pp,cum_rides_new_pp,...,np_counterfactual_rides,np_addl_rides,np_incre_rides_from_counterfactual,np_incre_rides_from_addl,total_incre_rides,cum_total_incre_rides,total_observed_rides,cum_total_observed_rides,cum_baseline_rides,progress_t_target
0,32,2025,2025-08-04,0,71141,71141,0.0023,163.248828,2514,2514,...,173.000000,0.000000,0.396987,0.000000,169.414746,169.414746,73828,73828,7.365859e+04,0.000237
1,33,2025,2025-08-11,1,69991,141132,0.0023,160.609897,3120,5634,...,356.000000,0.000000,0.816921,0.000000,168.586351,338.001097,73467,147295,1.469570e+05,0.000473
2,34,2025,2025-08-18,2,69816,210948,0.0023,160.208321,3787,9421,...,480.000000,0.000000,1.101467,0.000000,301.484312,639.485410,74083,221378,2.207385e+05,0.000896
3,35,2025,2025-08-25,3,67417,278365,0.0023,154.703282,4121,13542,...,478.000000,0.000000,1.096877,0.000000,165.256710,804.742119,72016,293394,2.925893e+05,0.001127
4,36,2025,2025-09-01,4,63875,342240,0.0023,146.575377,4062,17604,...,552.000000,0.000000,1.266687,0.000000,157.163225,961.905344,68489,361883,3.609211e+05,0.001347
5,37,2025,2025-09-08,5,78497,420737,0.0023,180.128804,6336,23940,...,741.000000,0.000000,1.700389,0.000000,585.412574,1547.317918,85574,447457,4.459097e+05,0.002167
6,38,2025,2025-09-15,6,81658,502395,0.0023,187.382420,7777,31717,...,893.656977,49.343023,2.050694,49.343023,836.206438,2383.524356,90378,537835,5.354515e+05,0.003339
7,39,2025,2025-09-22,7,76792,579187,0.0023,176.216303,7533,39250,...,901.192053,43.807947,2.067985,43.807947,614.057915,2997.582271,85270,623105,6.201074e+05,0.004199
8,40,2025,2025-09-29,8,74178,653365,0.0023,170.217899,7691,46941,...,714.935294,242.064706,1.640578,242.064706,895.945093,3893.527364,82826,705931,7.020375e+05,0.005454
9,41,2025,2025-10-06,9,79520,732885,0.0130,1020.493583,9493,56434,...,961.107692,271.892308,12.334057,271.892308,2309.517709,6203.045073,90246,796177,7.899740e+05,0.008689


In [91]:
incremental_all_rides.to_csv('incremtal_all_rides.csv',index= False)

In [ ]:
incremental_all_rides.to_hive_table(table_name='bp_incremental_all_rides', schema_name='shuninga', if_exists = 'replace')

{"ts": "2025-11-10T20:28:53.715Z", "uuid": "22f390c3-8775-4202-ab93-a794a012f509", "name": "lyft_data_toolkit.v1_to_v3.hive", "lvlname": "WARNING", "lvlno": 30, "pid": 97, "call_site": "code/venvs/venv/lib/python3.10/site-packages/lyft_data_toolkit/v1_to_v3/hive.py:32", "revision": "b8758af9c9", "msg": "DEPRECATION WARNING: to_hive_table method is outdated.Please use the new write_to_table function in the lyft_data_toolkit package."}
{"ts": "2025-11-10T20:28:54.170Z", "uuid": "d4ec9e47-9a4c-40eb-93c1-2ccc82ee147f", "name": "lyft_data_toolkit.db.read._read.Query", "lvlname": "WARNING", "lvlno": 30, "pid": 97, "call_site": "code/venvs/venv/lib/python3.10/site-packages/lyft_data_toolkit/db/read/_read.py:232", "revision": "b8758af9c9", "msg": "execution_context not supported for spark"}


Created Mozart command
{'mozart_id': 565782355, 'backend': 'spark', 'native_command_id': 'pending', 'job_url': 'https://sql.lyft.net/databases/mozart/queries/565782355', 'query': "SHOW TABLES FROM shuninga LIKE 'bp_incremental_all_rides'"}


/code/venvs/venv/lib/python3.10/site-packages/lyft_stats/client.py:330: UserWarning: [lyft_stats.client] stats dropped due to err=ConnectionResetError(104, 'Connection reset by peer')
  warnings.warn(f"[{__name__}] stats dropped due to {err=}")


### extra cells 

In [ ]:
# del lift_lookup

In [ ]:
# columns = [
#     "week_num", # begining week number of biweekly              
#     "year_num",  # beginning year number of the biweekly period             
#     "starting_week_date",    
#     "pre_existing_rider_lift", # get it from the tool
#     "new_rider_pp_lift"
# ]

# lift_lookup = pd.DataFrame(columns=columns)

# #  Function to append new biweekly data
# def add_biweekly_entry(df, week_num, year_num, starting_week_date, pre_existing_rider_lift, new_rider_pp_lift):
#     new_row = {
#         "week_num": int(week_num),
#         "year_num": int(year_num),
#         "starting_week_date": pd.to_datetime(starting_week_date),
#         "pre_existing_rider_lift": float(pre_existing_rider_lift),
#         "new_rider_pp_lift": float(new_rider_pp_lift)
#     }
#     return df.append(new_row, ignore_index=True)

# # do this biweekly to update the dataframe
# lift_lookup = add_biweekly_entry(lift_lookup, week_num=44, year_num=2025, starting_week_date="2025-11-02", pre_existing_rider_lift= 0.6, new_rider_pp_lift=1)

# # optional
# lift_lookup.to_csv("biweekly_lift_data.csv", index=False)

# lift_lookup